生データからラリーの表を作成し、
①勝因を出力
②対応についてのデータを出力

In [2]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import collections

In [3]:
folder = "puyopuyo_saikyo_league/stats/s1_day1_momoken_rein_2"
games = 57

In [4]:
#生データからラリーへの変換
#生データのcsvファイルから "Frame" "Player" "Rensa" "Honsen" "Ojama" "Ojama_detail" "Zenkeshi" "Continue" "Start" "Win"を参照


for i in range(games):
    #生データファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    N = len(df_game["Frame"])
    
    #初期化
    first_trigger_list = []
    
    rally = 0
    rally_times_list = []
    rally_winner_list = []
    
    rensa_list = []
    first_rensa_list = []
    max_rensa_list = []
    
    ojama_list = []

    honsen_flg_list = []
    
    sakiuchi_player_list = []
    honsen_player_tmp = []
    sakiuchi_rally_list = []
    honsen_rally_tmp = []

    second_flg_list = []
    #second_tmp[0]:本線発生したら1,それまでは0, [1]:先打ちしたプレイヤー(1or2)
    second_tmp = [0, 0]
    second_flg = 0
    
    for j in range(N):
        #エラーの時は例外処理
        if df_game["Player"].iloc[j] not in [1, 2]:
            pass
        else:
            #rensa_listに連鎖数を格納していき、ラリーの最初(first_trigger)と最大連鎖数(max_rensa)を取得
            rensa_list.append(df_game["Rensa"].iloc[j])
            
            #ラリー開始時の連鎖数を取得
            if df_game["Start"].iloc[j] == 1:
                first_trigger_list.append(df_game["Player"].iloc[j])

            #本線判定：最下段のぷよが消える(Honsen=1)かつ連鎖数が6以上の場合、本線と判定
            if (df_game["Honsen"].iloc[j] == 1)and(df_game["Rensa"].iloc[j] > 5):
                #本線を打ったプレイヤーとそのときのラリー数(最初が1)を格納
                honsen_player_tmp.append(df_game["Player"].iloc[j])
                honsen_rally_tmp.append(rally+1)
                
                #セカンド判定：先打ち時に[0](本線の存在有無)を1にし、[1](先打ちプレイヤー)を設定
                if second_tmp[0] == 0:
                    second_tmp[0] = 1
                    second_tmp[1] = df_game["Player"].iloc[j]
                else:
                    #本線がすでに存在する場合([0]=1)、先打ちプレイヤーと次に本線(最下段のぷよが消える)を打ったプレイヤーが同じで、
                    #その連鎖数が4以上であればセカンドと判定、second_flgを1にする
                    if (second_tmp[1] == df_game["Player"].iloc[j])&(df_game["Honsen"].iloc[j] == 1)&(df_game["Rensa"].iloc[j] > 3):
                        second_flg = 1

            #ラリー終了判定
            #ラリーが終了(おじゃまが降った)場合
            if df_game["Continue"].iloc[j] == 0:
                #ラリー数を増やし、rally_times_listに格納、初期化
                rally += 1
                rally_times_list.append(rally)
                rally = 0
                #おじゃまを降らせたプレイヤーをラリーの勝者として格納
                rally_winner_list.append(df_game["Ojama"].iloc[j])
                #先打ちしたプレイヤーがセカンドを打ったかどうか格納
                second_flg_list.append(second_flg)
                second_tmp = [0, 0]
                second_flg = 0
                #ラリーの最初の連鎖数、最大連鎖数を格納、初期化
                first_rensa_list.append(rensa_list[0])
                max_rensa_list.append(max(rensa_list))
                rensa_list = []
                #本線が存在する場合、本線の存在有無、先打ちしたプレイヤーと先打ちした時のラリー数を格納(本線がない場合は0)、初期化
                if (len(honsen_player_tmp)>0):
                    sakiuchi_player_list.append(honsen_player_tmp[0])
                    sakiuchi_rally_list.append(honsen_rally_tmp[0])
                    honsen_flg_list.append(1)
                else:
                    sakiuchi_player_list.append(0)
                    sakiuchi_rally_list.append(0)
                    honsen_flg_list.append(0)
                honsen_rally_tmp = []
                honsen_player_tmp = []
                #ラリーで発生したおじゃまの個数を格納
                #試合の最後でおじゃまの個数をカウントできなかった場合はおじゃま100個として別カウント
                ojama_list.append(df_game["Ojama_detail"].iloc[j])
                
            #ラリーが続く(おじゃまが発生しない)場合、ラリー数を増やす
            else:
                rally += 1
    
    #ラリーのデータフレーム作成
    df_rally = pd.DataFrame()
    df_rally["First_Trigger"] = pd.Series(first_trigger_list)
    df_rally["Rally_Times"] = pd.Series(rally_times_list)
    df_rally["Rally_Winner"] = pd.Series(rally_winner_list)
    df_rally["First_Rensa"] = pd.Series(first_rensa_list)
    df_rally["Max_Rensa"] = pd.Series(max_rensa_list)
    df_rally["Number_of_Ojama"] = pd.Series(ojama_list)
    df_rally["Honsen_Flg"] = pd.Series(honsen_flg_list)
    df_rally["Sakiuchi_Player"] = pd.Series(sakiuchi_player_list)
    df_rally["Sakiuchi_Rally_No"] = pd.Series(sakiuchi_rally_list)
    df_rally["Second_Flg"] = pd.Series(second_flg_list)
    df_rally["Winner"] = df_game["Win"]
    df_rally["Zenkeshi_Flg"] = df_game["Zenkeshi"]
    
    #ファイル出力
    filepath = folder + "/rally_" + str(i) + ".csv"
    df_rally.to_csv(filepath, encoding='utf_8_sig')

In [5]:
#勝因を探る

#－－－－－－ルールーーーーーー
#全消し戦→全消し戦
#本線バトルがない→対応
#本線が存在する
    #「先打ち」の直前に2段以上おじゃまを降らせたラリーがある→対応
    #「先打ち」したプレイヤーが勝利し、セカンドがある→セカンド
    #それ以外→先打ちor後打ち

game_honsen_sakiuchi_1P = 0
game_honsen_atouchi_1P = 0
game_honsen_sakiuchi_2P = 0
game_honsen_atouchi_2P = 0
game_second_1P = 0
game_second_2P = 0
game_taiou_1P = 0
game_taiou_2P = 0
game_zenkeshi_1P = 0
game_zenkeshi_2P = 0
game_1P = 0
game_2P = 0

def my_index_multi(l, x):
    return [i for i, _x in enumerate(l) if _x == x]

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/rally_" + str(i) + ".csv"
    df_rally = pd.read_csv(filepath)
    N = len(df_rally["First_Trigger"])
    
    #全消し戦の場合は全消し戦の判定
    if df_rally["Zenkeshi_Flg"].iloc[0] == 1:
        if df_rally["Winner"].iloc[0] == 1:
            game_zenkeshi_1P += 1
        else:
            game_zenkeshi_2P += 1
    
    #本線がある場合
    elif (1 in list(df_rally["Honsen_Flg"])):
        honsen_index = min(my_index_multi(list(df_rally["Honsen_Flg"]), 1))
        #1Pが勝った試合の場合
        if df_rally["Winner"].iloc[0] == 1:
            #1Pが先打ちの場合
            #セカンドor対応(直前に2段以上のおじゃまを降らせた場合のみ)or先打ち
            if df_rally["Sakiuchi_Player"].iloc[honsen_index] == 1:
                if df_rally["Second_Flg"].iloc[honsen_index] == 1:
                    game_second_1P += 1
                elif (df_rally["Number_of_Ojama"].iloc[honsen_index-1] >= 13)&(df_rally["Rally_Winner"].iloc[honsen_index-1] == 1)&(honsen_index!=0):
                    game_taiou_1P += 1
                else:
                    game_honsen_sakiuchi_1P += 1
            #1Pが先打ちでない場合→後打ち
            else:
                game_honsen_atouchi_1P += 1
        #2Pが勝った場合の試合
        else:
            #2Pが先打ちの場合
            #セカンドor対応(直前に2段以上のおじゃまを降らせた場合のみ)or先打ち
            if df_rally["Sakiuchi_Player"].iloc[honsen_index] == 2:
                if df_rally["Second_Flg"].iloc[honsen_index] == 1:
                    game_second_2P += 1
                elif (df_rally["Number_of_Ojama"].iloc[honsen_index-1] >= 13)&(df_rally["Rally_Winner"].iloc[honsen_index-1] == 2)&(honsen_index!=0):
                    game_taiou_2P += 1
                else:
                    game_honsen_sakiuchi_2P += 1
            #2Pが先打ちでない場合→後打ち
            else:
                game_honsen_atouchi_2P += 1
    
    #本線がない場合→対応
    else:
        if df_rally["Winner"].iloc[0] == 1:
            game_taiou_1P += 1
        else:
            game_taiou_2P += 1
    
    #勝利数
    if df_rally["Winner"].iloc[0] == 1:
        game_1P += 1
    elif df_rally["Winner"].iloc[0] == 2:
        game_2P += 1
    else:
        pass

#DataFrame作成
cause_1P = [game_honsen_sakiuchi_1P, game_second_1P, game_honsen_atouchi_1P, game_taiou_1P, game_zenkeshi_1P, game_1P]
cause_2P = [game_honsen_sakiuchi_2P, game_second_2P, game_honsen_atouchi_2P, game_taiou_2P, game_zenkeshi_2P, game_2P]
index_list = ["本線先打ち", "本線先打ち+セカンド", "本線後打ち", "対応", "全消し戦", "勝利数"]
    

df_cause = pd.DataFrame(list(zip(cause_1P, cause_2P)), columns=["1P", "2P"],
                               index=index_list)   
df_cause

,1P,2P
本線先打ち,13,10
本線先打ち+セカンド,0,0
本線後打ち,5,3
対応,7,9
全消し戦,5,5
勝利数,30,27


In [7]:
#対応

#単発(ラリー発生せず)の回数をカウント
tanpatu_1P = 0
tanpatu_2P = 0
#対応のカウント
#①相手に先打ちさせた場合
taiou_sakiuchi_1P = 0
taiou_sakiuchi_2P = 0
#(①'置き去り本線)
okizari_1P = 0
okizari_2P = 0
#②おじゃまが12個未満のとき
taiou_small_1P = 0
taiou_small_2P = 0
#③おじゃまが12個以上の時
taiou_large_1P = 0
taiou_large_2P = 0
#④エラー
taiou_error_1P = 0
taiou_error_2P = 0

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/rally_" + str(i) + ".csv"
    df_rally = pd.read_csv(filepath)
    N = len(df_rally["Rally_Winner"])
    
    
    for j in range(N):
        #単発(Max_Rensa=1)のみのラリー
        if (df_rally["Max_Rensa"].iloc[j] == 1):
            if df_rally["Rally_Winner"].iloc[j] == 1:
                tanpatu_1P += 1
            else:
                tanpatu_2P += 1
        
        #本戦が存在する場合     
        elif (df_rally["Honsen_Flg"].iloc[j] == 1):
            #"Honsen_Rally"が2以上(相手の連鎖に対して本線を打った)の場合、相手に先打ちさせられた判定
            if df_rally["Sakiuchi_Rally_No"].iloc[j]>1:
                if df_rally["Sakiuchi_Player"].iloc[j] == 1:
                    taiou_sakiuchi_2P += 1
                else:
                    taiou_sakiuchi_1P += 1
            #いきなり本線を打った場合、置き去り本線(集計には使用せず)
            else:
                if df_rally["Sakiuchi_Player"].iloc[j] == 1:
                    okizari_1P += 1
                else:
                    okizari_2P += 1        
        
        #それ以外は対応
        #降ったおじゃまぷよの数で分類
        
        else:
            #1Pが制したラリー
            if df_rally["Rally_Winner"].iloc[j] == 1:
                #ゲームの最後等、降ったおじゃまの個数を画像から判定できない場合(Number_of_Ojama=100として例外扱いにしています)
                #→ラリー中のMax_Rensaが3連鎖以上なら2段以上、2連鎖以下なら2段未満にカウント
                if df_rally["Number_of_Ojama"].iloc[j] == 100:
                    if df_rally["Max_Rensa"].iloc[j] >= 3:
                        taiou_large_1P += 1
                    else:
                        taiou_small_1P += 1
                #おじゃまの個数が画像から判定できる場合は個数で分類
                elif df_rally["Number_of_Ojama"].iloc[j] < 12:
                    taiou_small_1P += 1
                elif df_rally["Number_of_Ojama"].iloc[j] >= 12:
                    taiou_large_1P += 1
                else:
                    taiou_error_1P += 1
            #2Pが制したラリー
            else:
                #ゲームの最後等、降ったおじゃまの個数を画像から判定できない場合(Number_of_Ojama=100として例外扱いにしています)
                #→ラリー中のMax_Rensaが3連鎖以上なら2段以上、2連鎖以下なら2段未満にカウント
                if df_rally["Number_of_Ojama"].iloc[j] == 100:
                    if df_rally["Max_Rensa"].iloc[j] >= 3:
                        taiou_large_2P += 1
                    else:
                        taiou_small_2P += 1
                #おじゃまの個数が画像から判定できる場合は個数で分類
                elif df_rally["Number_of_Ojama"].iloc[j] < 12:
                    taiou_small_2P += 1
                elif df_rally["Number_of_Ojama"].iloc[j] >= 12:
                    taiou_large_2P += 1
                else:
                    taiou_error_2P += 1
                    print(i)

#集計
taiou_total_1P = taiou_sakiuchi_1P + taiou_small_1P + taiou_large_1P + taiou_error_1P
taiou_total_2P = taiou_sakiuchi_2P + taiou_small_2P + taiou_large_2P + taiou_error_2P

#DataFrame作成
taiou_list_1P = [tanpatu_1P, taiou_total_1P, taiou_sakiuchi_1P, taiou_small_1P, taiou_large_1P, taiou_error_1P, 
                 str(round(100*taiou_total_1P/(taiou_total_1P+taiou_total_2P), 1))]
taiou_list_2P = [tanpatu_1P, taiou_total_2P, taiou_sakiuchi_2P, taiou_small_2P, taiou_large_2P, taiou_error_2P, 
                 str(round(100*taiou_total_2P/(taiou_total_1P+taiou_total_2P), 1))]
index_list = ["単発ラリー勝利数", "対応ラリー勝利数", "内訳①本線を打たせた", "内訳②おじゃま2段未満", "内訳③おじゃま2段以上", "内訳④エラー", "対応勝率"]
    

df_taiou = pd.DataFrame(list(zip(taiou_list_1P, taiou_list_2P)), columns=["1P", "2P"],
                               index=index_list)   
df_taiou

,1P,2P
単発ラリー勝利数,14,14
対応ラリー勝利数,35,43
内訳①本線を打たせた,17,20
内訳②おじゃま2段未満,9,16
内訳③おじゃま2段以上,9,7
内訳④エラー,0,0
対応勝率,44.9,55.1
